# make_tables.ipynb
**Purpose:** Generate LaTeX tables used in the manuscript (e.g., tuned hyperparameters, feature rankings, thresholds).  
**Inputs:** Excel/CSV outputs from Stage 1 and Stage 2 tuning scripts.  
**Outputs:** LaTeX `.txt` table files.

**Part of repository:** `3_Graphs_Tables`  
**Reproducibility:** This notebook is deterministic given the provided model output files.


In [23]:
import pandas as pd
import ast
import re
from make_outputs import format_label, load_selected_models
from pathlib import Path
import os

In [10]:
SUBREDDIT_LABELS = {
    "conspiracy": "r/Conspiracy",
    "crypto": "r/CryptoCurrency",
    "politics": "r/politics",
}

col_digits = {'colsample_bytree': 3, 'learning_rate': 3, 'max_depth':0, 'min_child_samples':0,
       'num_leaves':0, 'reg_alpha':3, 'reg_lambda':3, 'subsample':3}
int_cols = ['Features', 'max_depth', 'min_child_samples', 'num_leaves']

In [9]:
def parse_cw_string(s):
    cleaned = re.sub(r"np\.float64\(([^)]+)\)", r"\1", s)
    return ast.literal_eval(cleaned)

def round_dec(val, d=2):
    return round(val, d)

In [4]:

def clean_tabular(tabular):
    tabular = r"""\hline
""" + tabular
    tabular = tabular.replace(r"\\", r"\\ \hline")
    to_repl = [r'\toprule', r'\midrule', r'\bottomrule']
    for s in to_repl:
        tabular = tabular.replace(s, "")
    return tabular
        
def build_supp_table(title: str, tabular: str, caption: str, n=None) -> str:
    """
    Build a supplementary-materials table:

    \paragraph*{SN Table.}{\bf TITLE}
        \label{SN-Table}
        \begin{table}[!ht]
            \centering
    LATEX_TABLE
        \end{table}
    CAPTION
    """
    if n is None:
        n=""
    tabular = clean_tabular(tabular)

    return (
        rf"\paragraph*{{S{n} Table.}}{{\bf {title}}}" "\n"
        rf"\label{{S{n}-Table}}" "\n"
        r"\begin{table}[!ht]" "\n"
        r"    \centering" "\n"
        f"{tabular}\n"
        r"\end{table}" "\n"
        f"{caption}"
    )

def build_main_table(title, latex_table, caption, label):
    """
    Construct a LaTeX table environment with consistent structure.

    Parameters
    ----------
    title : str
        Title displayed in bold in the caption.
    latex_table : str
        The tabular environment as a LaTeX string.
    caption : str
        Additional caption text placed under the table.
    label : str
        Label name (without 'tab:').

    Returns
    -------
    str
        Full LaTeX table environment string.
    """

    # Clean unwanted booktabs commands
    latex_table = clean_tabular(latex_table)

    # Build final LaTeX table
    table = f"""
\\begin{{table}}[!ht]
\\centering
\\caption{{\\bf {title}}}
{latex_table}

\\begin{{flushleft}}{caption}\\end{{flushleft}}
\\label{{tab:{label}}}
\\end{{table}}
"""

    return table.strip()


In [14]:
eval_filepath = f"../../Publication_Outputs/1_Thread_Start/metrics/table_metrics.xlsx"
mcc_dfs = []
for sub in SUBREDDIT_LABELS:

    df = pd.read_excel(eval_filepath, sheet_name=f"{sub}_test").rename(columns={"n_feats": "Features"})

    mcc_df = df[['Features']].copy()
    mcc_df["MCC"] = df["MCC"].map(lambda x: f"{x:.4f}") + " " + df["MCC CI"].astype(str)

    mcc_df = mcc_df.set_index("Features")

    mcc_dfs.append(mcc_df[['MCC']].rename(columns={"MCC": f"{SUBREDDIT_LABELS[sub]}"}))

mcc_df = pd.concat(mcc_dfs, axis=1).reset_index()

latex_str = mcc_df.to_latex(
    index=False,                    # don’t print the row index
    header=True,
    float_format="%.4f",
    column_format="|l|r|r|r|",      # LaTeX alignment (1 left + 8 right)
    escape=True                    # so underscores in col names are not escaped weirdly
)

caption = r"Test-set MCC values (with 95\% bootstrap confidence intervals) for the tuned thread-start classifiers across the three subreddits. Each row corresponds to a model trained using the top-$n$ features. Higher MCC indicates better discrimination between classes."
title = f"Test-set MCC for the thread-start prediction models."
full_latex = build_supp_table(title, latex_str, caption)
# Save to file
with open(f"../../Publication_Outputs/1_Thread_Start/metrics/test_mccs.txt", "w") as f:
    f.write(full_latex)

In [20]:
s1_mods = load_selected_models(Path("../../Publication_Outputs/1_Thread_Start/s1_mods.txt"))
eval_filepath = f"../../Publication_Outputs/1_Thread_Start/metrics/table_metrics.xlsx"
rows = []
for sub in SUBREDDIT_LABELS:

    df = pd.read_excel(eval_filepath, sheet_name=f"{sub}_test").rename(columns={"n_feats": "Features"})
    df = df.set_index("Features")
    metric_cols = [c for c in df.columns if "CI" not in c]
    new_df = df[metric_cols].copy()
    for col in metric_cols:
        new_df[col] = df[col].map(lambda x: f"{x:.4f}") + " " + df[f"{col} CI"].astype(str)
    
    mod_row = new_df.iloc[s1_mods[sub]-1,:]
    mod_row['Subreddit'] = SUBREDDIT_LABELS[sub]
    mod_row['Features'] = s1_mods[sub]
    rows.append(mod_row)

df = pd.concat(rows, axis=1)
# --- Replace column headers with subreddit names ---
subreddit_names = df.loc["Subreddit"].tolist()
df.columns = subreddit_names

# --- Move 'Features' row to top ---
features_row = df.loc["Features"].to_frame().T

# Drop Subreddit + Features rows
df = df.drop(["Subreddit", "Features"])

# Insert Features at top
df = pd.concat([features_row, df])


tabular = df.to_latex(
    index=True,                    # don’t print the row index
    header=True,
    float_format="%.4f",
    column_format="|l|r|r|r|",      # LaTeX alignment (1 left + 8 right)
    escape=True                    # so underscores in col names are not escaped weirdly
)

caption = r"The thread start model test set MCC, AUC, F1 score, balanced accuracy, and stalled started precision and recall scores with their corresponding 95\% confidence intervals computed with 1000 bootstrap resamples."
title = f"Test set performance of the selected thread start models for each subreddit"
full_latex = build_main_table(title, tabular, caption, 's1-mods')
# Save to file
with open(f"../../Publication_Outputs/1_Thread_Start/metrics/selected_mods.txt", "w") as f:
    f.write(full_latex)

/tmp/ipykernel_185827/3239559225.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mod_row['Subreddit'] = SUBREDDIT_LABELS[sub]
/tmp/ipykernel_185827/3239559225.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mod_row['Subreddit'] = SUBREDDIT_LABELS[sub]
/tmp/ipykernel_185827/3239559225.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mod_row['Features'] = s1_mods[sub]
/tmp/ipykernel_185827/3239559225.py:15: SettingWithCopyW

In [12]:
hyperparam_dfs = {}
sheet_name = "all_hyperparams"
for subreddit in SUBREDDIT_LABELS:
    eval_filepath = f"../../Outputs/1_thread_start/{subreddit}/4_model/evaluation.xlsx"
    df = pd.read_excel(eval_filepath, sheet_name=sheet_name)
    
    # Ensure correct column names
    df.columns = ["Features", "Parameter", "Value"]

    # Forward-fill N_feats to propagate block identifiers
    df["Features"] = df["Features"].ffill().astype(int)

    # Pivot into the desired wide format
    table = df.pivot(index="Features", columns="Parameter", values="Value")
    for col in table.columns:
        table[col] = table[col].apply(round_dec, d=col_digits[col])
    for col in [x for x in table.columns if x in int_cols]:
        table[col] = table[col].astype(int)
    hyperparam_dfs[subreddit] = table.copy()

    table = table.reset_index()  # if N_feats was the index

    latex_str = table.to_latex(
        index=False,                    # don’t print the row index
        header=True,
        float_format="%.3f",         # control float precision
        column_format="|l|r|r|r|r|r|r|r|r|",      # LaTeX alignment (1 left + 8 right)
        #caption=f"{SUBREDDIT_LABELS[subreddit]} tuned thread start LightGBM hyperparameters by number of features.",
        #label=f"tab:s1-{subreddit}-hyperparams",
        escape=True                    # so underscores in col names are not escaped weirdly
    )
    
    caption = f"Optimal LightGBM tree hyperparameters selected via cross-validated Optuna/TPE search for each number of features for {SUBREDDIT_LABELS[subreddit]}. Values represent cross-fold aggregated hyperparameters, using the mode for integer parameters and the mean for continuous parameters. These configurations were used for the final thread start model evaluation."
    title = f"{subreddit} tuned thread start LightGBM hyperparameters by number of features."
    full_latex = build_supp_table(title, latex_str, caption)
    # Save to file
    with open(f"../../Publication_Outputs/1_Thread_Start/tuning_outputs/{subreddit}_hparams.txt", "w") as f:
        f.write(full_latex)

outfile = f"../../Publication_Outputs/1_Thread_Start/tuning_outputs/hyperparams.xlsx"
with pd.ExcelWriter(outfile) as writer:
    for sub, df in hyperparam_dfs.items():
        df.to_excel(writer, sheet_name=sub)


In [6]:
model_threshold_dfs = {}
sheet_name = "all_params"
for subreddit in SUBREDDIT_LABELS:
    eval_filepath = f"../../Outputs/1_thread_start/{subreddit}/4_model/evaluation.xlsx"
    df = pd.read_excel(eval_filepath, sheet_name=sheet_name)
    
    # Ensure correct column names
    df.columns = ["Features", "param_index", "Parameter", "Value"]

    # Forward-fill N_feats to propagate block identifiers
    df["Features"] = df["Features"].ffill().astype(int)

    # Pivot into the desired wide format
    table = df.pivot(index="Features", columns="Parameter", values="Value")

    model_threshold_dfs[subreddit] = table.copy()[['model_threshold']]


In [7]:
pd.concat(model_threshold_dfs, axis=1).to_csv(f"../../Publication_Outputs/1_Thread_Start/metrics/model_thresholds.xlsx")

In [ ]:
s2_cw_dfs = {}
sheet_name = "params"
for subreddit in SUBREDDIT_LABELS:
    eval_filepath = f"../../Outputs/2_thread_size/{subreddit}/2_tuning/tuning_outputs.xlsx"
    df = pd.read_excel(eval_filepath, sheet_name=sheet_name)
    
    # Ensure correct column names
    df = df[['n_feats', 'final_class_weights']].rename(columns={
        "final_class_weights": "cws"
    })

    s2_cw_dfs[subreddit] = df

In [36]:
CLASS_NAMES = ["Stalled", "Small", "Medium", "Large"]

In [ ]:
outfile = f"../../Publication_Outputs/2_Thread_Size/tuning_outputs/class_weights.xlsx"

with pd.ExcelWriter(outfile) as writer:
    for sub, df in s2_cw_dfs.items():
        expanded = pd.json_normalize(df["cws"].apply(parse_cw_string))
        df = df.join(expanded)[["n_feats", 0,1,2,3]]
        ratios = df[['n_feats']].copy()
        for col in [0,1,2,3]:
            ratios[CLASS_NAMES[col]] = df[col]/df[0]
        ratios.to_excel(writer, index=False, sheet_name=sub)
        


In [ ]:
s2_feat_dfs = {}
sheet_name = "feature_importances"
for subreddit in SUBREDDIT_LABELS:
    eval_filepath = f"../../Outputs/2_thread_size/{subreddit}/2_tuning/tuning_outputs.xlsx"
    df = pd.read_excel(eval_filepath, sheet_name=sheet_name)
    
    # Ensure correct column names
    df = df[['feature', 'mean_importance', 'mean_split', 'mean_gain']].rename(columns={
        'feature': 'Feature',
        'mean_importance': 'Scaled',
        'mean_split': 'Split',
        'mean_gain': 'Gain'
    })
    df['Feature'] = df['Feature'].apply(format_label)
    df['Scaled'] = df["Scaled"].apply(round_dec, d=4)
    df['Split'] = df["Split"].apply(round_dec, d=0)
    df['Gain'] = df["Gain"].apply(round_dec, d=0)

    s2_feat_dfs[subreddit] = df

outfile = f"../../Publication_Outputs/2_Thread_Size/tuning_outputs/feature_importances.xlsx"

with pd.ExcelWriter(outfile) as writer:
    for sub, df in s2_feat_dfs.items():
        df.to_excel(writer, sheet_name=sub)

In [24]:
os.makedirs("../../Publication_Outputs/2_Thread_Size/tuning_outputs/", exist_ok=True)

In [25]:
hyperparam_dfs = {}
sheet_name = "hyperparams"
for subreddit in SUBREDDIT_LABELS:
    eval_filepath = f"../../Outputs/2_thread_size/{subreddit}/4_model/evaluation.xlsx"
    df = pd.read_excel(eval_filepath, sheet_name=sheet_name)
    # Ensure correct column names
    df.columns = ["Parameter", "Value", "Features"]
    # Forward-fill N_feats to propagate block identifiers
    df["Features"] = df["Features"].ffill().astype(int)

    # Pivot into the desired wide format
    table = df.pivot(index="Features", columns="Parameter", values="Value")
    for col in table.columns:
        table[col] = table[col].apply(round_dec, d=col_digits[col])
    for col in [x for x in table.columns if x in int_cols]:
        table[col] = table[col].astype(int)
    hyperparam_dfs[subreddit] = table.copy()

    table = table.reset_index()  # if N_feats was the index
    

    latex_str = table.to_latex(
        index=False,                    # don’t print the row index
        header=True,
        float_format="%.3f",
        column_format="|l|r|r|r|r|r|r|r|r|",      # LaTeX alignment (1 left + 8 right)
        #caption=f"{SUBREDDIT_LABELS[subreddit]} tuned thread start LightGBM hyperparameters by number of features.",
        #label=f"tab:s1-{subreddit}-hyperparams",
        escape=True                    # so underscores in col names are not escaped weirdly
    )
    
    caption = f"Optimal LightGBM tree hyperparameters selected via cross-validated Optuna/TPE search for each number of features for {SUBREDDIT_LABELS[subreddit]}. Values represent cross-fold aggregated hyperparameters, using the mode for integer parameters and the mean for continuous parameters. These configurations were used for the final thread size model evaluation."
    title = f"{SUBREDDIT_LABELS[subreddit]} tuned thread size LightGBM hyperparameters by number of features."
    full_latex = build_supp_table(title, latex_str, caption)
    # Save to file
    with open(f"../../Publication_Outputs/2_Thread_Size/tuning_outputs/{subreddit}_hparams.txt", "w") as f:
        f.write(full_latex)

In [26]:
eval_filepath = f"../../Publication_Outputs/2_Thread_Size/metrics/table_metrics.xlsx"
mcc_dfs = []
for sub in SUBREDDIT_LABELS:

    df = pd.read_excel(eval_filepath, sheet_name=f"{sub}_test").rename(columns={"n_feats": "Features"})

    mcc_df = df[['Features']].copy()
    mcc_df["MCC"] = df["MCC"].map(lambda x: f"{x:.4f}") + " " + df["MCC CI"].astype(str)

    mcc_df = mcc_df.set_index("Features")

    mcc_dfs.append(mcc_df[['MCC']].rename(columns={"MCC": f"{SUBREDDIT_LABELS[sub]}"}))

mcc_df = pd.concat(mcc_dfs, axis=1).reset_index()

latex_str = mcc_df.to_latex(
    index=False,                    # don’t print the row index
    header=True,
    float_format="%.4f",
    column_format="|l|r|r|r|",      # LaTeX alignment (1 left + 8 right)
    escape=True                    # so underscores in col names are not escaped weirdly
)

caption = r"Test-set MCC values (with 95\% bootstrap confidence intervals) for the tuned thread-size classifiers across the three subreddits. Each row corresponds to a model trained using the top-$n$ features. Higher MCC indicates better discrimination between classes."
title = f"Test-set MCC for the thread-size prediction models."
full_latex = build_supp_table(title, latex_str, caption)
# Save to file
with open(f"../../Publication_Outputs/2_Thread_Size/metrics/test_mccs.txt", "w") as f:
    f.write(full_latex)

In [27]:
s2_mods = load_selected_models(Path("../../Publication_Outputs/2_Thread_Size/s2_mods.txt"))
eval_filepath = f"../../Publication_Outputs/2_Thread_Size/metrics/table_metrics.xlsx"
rows = []
for sub in SUBREDDIT_LABELS:

    df = pd.read_excel(eval_filepath, sheet_name=f"{sub}_test").rename(columns={"n_feats": "Features"})
    df = df.set_index("Features")
    metric_cols = [c for c in df.columns if "CI" not in c]
    new_df = df[metric_cols].copy()
    for col in metric_cols:
        new_df[col] = df[col].map(lambda x: f"{x:.4f}") + " " + df[f"{col} CI"].astype(str)
    
    mod_row = new_df.iloc[s2_mods[sub]-1,:]
    mod_row['Subreddit'] = SUBREDDIT_LABELS[sub]
    mod_row['Features'] = s2_mods[sub]
    rows.append(mod_row)

df = pd.concat(rows, axis=1)
# --- Replace column headers with subreddit names ---
subreddit_names = df.loc["Subreddit"].tolist()
df.columns = subreddit_names

# --- Move 'Features' row to top ---
features_row = df.loc["Features"].to_frame().T

# Drop Subreddit + Features rows
df = df.drop(["Subreddit", "Features"])

# Insert Features at top
df = pd.concat([features_row, df])


tabular = df.to_latex(
    index=True,                    # don’t print the row index
    header=True,
    float_format="%.4f",
    column_format="|l|r|r|r|",      # LaTeX alignment (1 left + 8 right)
    escape=True                    # so underscores in col names are not escaped weirdly
)

caption = r"The thread size model test set MCC, AUC, F1 score, balanced accuracy, and stalled, small, medium and large precision and recall scores with their corresponding 95\% confidence intervals computed with 1000 bootstrap resamples."
title = f"Test set performance of the selected thread size models for each subreddit"
full_latex = build_main_table(title, tabular, caption, 's2-mods')
# Save to file
with open(f"../../Publication_Outputs/2_Thread_Size/metrics/selected_mods.txt", "w") as f:
    f.write(full_latex)

/tmp/ipykernel_185827/2292598750.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mod_row['Subreddit'] = SUBREDDIT_LABELS[sub]
/tmp/ipykernel_185827/2292598750.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mod_row['Subreddit'] = SUBREDDIT_LABELS[sub]
/tmp/ipykernel_185827/2292598750.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mod_row['Features'] = s2_mods[sub]
/tmp/ipykernel_185827/2292598750.py:15: SettingWithCopyW